# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#import matplotlib.pyplot as pltimport gmaps
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt
import pprint
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from matplotlib import pyplot as plt
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/Weather_Data.csv")
weather_df = pd.DataFrame(weather_data)
weather_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Souillac,-20.52,57.52,73.40,64,40,17.22
1,Labuan,5.28,115.24,87.40,66,75,3.36
2,Ishigaki,24.34,124.16,77.00,100,75,17.22
3,Villaviciosa,43.48,-5.44,56.73,82,100,4.00
4,Araouane,18.90,-3.53,96.94,24,7,6.91
...,...,...,...,...,...,...,...
495,Paita,-5.09,-81.11,65.35,85,7,11.16
496,Darhan,49.49,105.92,55.06,66,100,0.96
497,Bjørnevatn,69.67,29.98,57.20,58,75,33.33
498,Nador,35.17,-2.93,74.23,60,3,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Generate Heat Map Based on Humidity Percentages

# Establish API Key
gmaps.configure(api_key=g_key)

# Define Heat Map Locations and Weights
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity (%)"].astype(float)

# Create Heat Map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=True, max_intensity=100,
                                point_radius = 7)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow Down Personal Optimal Weather Conditions
optimal_weather = weather_df[(weather_df["Temperature (F)"] > 65) & (weather_df["Temperature (F)"] < 75)]
optimal_weather = optimal_weather[(optimal_weather["Humidity (%)"] < 50)]

# Create New DataFrame to Store Hotel Data
optimal_df = optimal_weather[["City", "Latitude", "Longitude"]].reset_index(drop=True)
optimal_df["Hotel Name"] = ""
optimal_df["Hotel Address"] = ""
optimal_df["Hotel Latitude"] = ""
optimal_df["Hotel Longitude"] = ""
optimal_df


,City,Latitude,Longitude,Hotel Name,Hotel Address,Hotel Latitude,Hotel Longitude
0,Hakkâri Province,37.58,44.17,,,,
1,Zhigansk,66.77,123.37,,,,
2,Melfi,41.00,15.65,,,,
3,Henties Bay,-22.12,14.28,,,,
4,Carnarvon,-24.87,113.63,,,,
5,Zaysan,47.47,84.87,,,,
6,Chiredzi,-21.05,31.67,,,,
7,Aykhal,66.00,111.50,,,,
8,Kambove,-10.88,26.60,,,,
9,Xining,36.62,101.77,,,,


In [5]:

# Establish Parameters For Hotel Search
params = {
    'types': 'lodging',
    'radius': 5000,
    "key": g_key
}

# Iterate Each City Location to Obtain Hotel Data
for index, row in optimal_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
            
    try:
        optimal_df.loc[index, "Hotel Name"] = name_address['results'][0]['name']
        optimal_df.loc[index, "Hotel Address"] = name_address['results'][0]['vicinity']
        optimal_df.loc[index, "Hotel Latitude"] = name_address['results'][0]['geometry']['location']['lat']
        optimal_df.loc[index, "Hotel Longitude"] = name_address['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("N/A")
        
optimal_df

N/A
N/A
N/A
N/A
N/A


,City,Latitude,Longitude,Hotel Name,Hotel Address,Hotel Latitude,Hotel Longitude
0,Hakkâri Province,37.58,44.17,,,,
1,Zhigansk,66.77,123.37,Gostinichnyy Kompleks Aylgy,"Октябрьская,д, 42а",66.7534,123.383
2,Melfi,41.00,15.65,HOTEL TERME DI RAPOLLA,"Via Melfi, 170, Rapolla",40.9784,15.6711
3,Henties Bay,-22.12,14.28,Fishermans Guest House,Namibia,-22.1336,14.2911
4,Carnarvon,-24.87,113.63,Hospitality Carnarvon,"6 West Street, South Carnarvon",-24.8923,113.655
5,Zaysan,47.47,84.87,Gostinitsa Aray,"Астана, 58 Street, Zaysan",47.4722,84.8706
6,Chiredzi,-21.05,31.67,The Nesbitt Arms,"238 Marula Drive, Chiredzi",-21.0455,31.668
7,Aykhal,66.00,111.50,,,,
8,Kambove,-10.88,26.60,Couvant des soeurs catholiques,"Unnamed Road, Kambove",-10.8742,26.5976
9,Xining,36.62,101.77,Super Leisure Hotel,"36 Wusi West Road, Xining",36.6377,101.744


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in optimal_df.iterrows()]
locations = optimal_df[["Latitude", "Longitude"]]


In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))